# Pattern Recognition pipeline
The entire pipeline should be runnable from here.

### Genes data
**Raw genes data**
- Data is expected to be in *raw_data/genes/data.csv*
- Labels are expected to be in *raw_data/genes/labels.csv*

### Image data
**Raw image data**
- Animal images are expected to be in *raw_data/BigCats/[Animal]/*

### Feature selection
- Feature selection functions can be found in *feature_selection/[function].py*

### Classification
- Classification functions can be found in *classification/[function].py*

### Clustering
- Clustering functions can be found in *clustering/[function].py*

# Code


In [ ]:
## Import ALL required imports

# Ignore SKlearn's deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Data imports
from raw_data.data_functions import load_num_data, load_img_data

# Feature selection imports
from feature_selection.pca import pca

# Classification imports
from classification.knn import knn, knn_gridsearch

## Load Data

In [ ]:
num_data, num_labels = load_num_data()
img_data, img_labels = load_img_data()

## Feature Extraction

### PCA

In [5]:
pca_data = pca(num_data)

### MI

### SIFT

### Fourier

## Classification

### KNN

#### 80/20 split

In [ ]:
knn_gridsearch(pca_data, labels, [1,2])

#### K-fold

### Naive Bayes

#### 80/20 split

#### K-fold

### Decision Trees

#### 80/20 split

#### K-fold

### Logistic Regression

#### 80/20 split

#### K-fold

### SVM

#### 80/20 split

#### K-fold

## Clustering

### K-Means

#### 80/20 split

#### K-fold

### Fuzzy C-Means

#### 80/20 split

#### K-fold

-----------------------------------------------------
*end*

## Mutual Information and SVM

### For Genes data

In [1]:

# Data imports
from raw_data.data_load_MI_SVM import load_genes_data#, load_img_data

# Feature selection imports
from feature_selection.MI import MI_feature_select

# Classification imports
from classification.svm import svm_genes

In [2]:
featr_nohead,lbs_noH=load_genes_data()
Feat_slct=MI_feature_select(featr_nohead,lbs_noH,thre=0.5)
svm_genes(Feat_slct,lbs_noH)

load data start
load data succ
feature selection start


C:\Users\yunmeng\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


478 features whose MI score over threshold are selected from 20531 features
feature selection end
svm train start
accuracy on the training subset:1.000
accuracy on the test subset:0.994


C:\Users\yunmeng\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### For Image data

In [1]:
# Data imports
from raw_data.data_load_MI_SVM import getImageData

# Feature selection imports
from feature_selection.MI import MI_feature_select

# Classification imports
from classification.svm import svm_images

In [5]:
directory ="../Data-211216/Data/BigCats/" 
feature_array,label_array = getImageData(directory)
feature_array=MI_feature_select(feature_array, label_array,thre=0.1) # does not work well 
svm_images(feature_array,label_array)

data load succ
feature selection start
580 features whose MI score over threshold are selected from 10000 features
feature selection end
svm train start
Best model using {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
new model saved
accuracy on the training subset:0.963
accuracy on the test subset:0.353
success
